In [1]:
from MQGO import data, main, ML_engine, optimizer, QC_engine 
from deepmd.calculator import DP
import subprocess
import numpy as np
import json

2022-05-05 17:26:43.944237: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-05 17:26:43.944281: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
d_steps = [5000]
tot_steps = [6000]
subprocess.run('cp CdSe_2.xyz CdSe.xyz', shell=True)

index = 0
for ii in range(0,len(d_steps)):
    for jj in range(0,len(tot_steps)):
        js_setting = {}
        with open('./input0.json', 'r') as file:
            json_setting = json.load(file)
            
        json_setting['learning_rate']['decay_steps'] = d_steps[ii]
        # json_setting['learning_rate']['start_lr'] = 0.001
        json_setting['training']['numb_steps'] = tot_steps[jj]

        with open('./input.json', 'w') as js:
            json.dump(json_setting, js, indent=4)
        
        setting = {'basis':'def2-tzvp', 'ecp':'lanl2dz', 'spin':0, 'xc':'pbe', 'restricted':True, 'verbose':3, 'max_cycle':500}
        geo = main.MLgeomopt(qc_engine='pyscf', ml_engine='deepmd', work_path='./', xyz_path='./CdSe.xyz',**setting)
        geo.opt_algorithm = 'lbfgs'
        geo.debug = True
        geo.have_init_qc_data = False
        geo.kernel()
        
        Opt = optimizer.Optimizer(xyz_path="CdSe_2.xyz",
									work_path="./",
									ml_engine='deepmd',
									outter_cycle=1,
									algorithm='pyberny',
									max_opt_cycle=200)
        atom0 = Opt.ase_read_xyz()
        atom0.calc = DP(model='graph.pb')
        print(atom0.get_positions())
        force0_ml = atom0.get_forces()

        forces = np.loadtxt('raw/force.raw')
        force0_qc = forces[0].reshape((6,3))

        delta = force0_qc-force0_ml
        with open('debug_diff_force.txt', 'a') as file:
            file.write('%24.18f\n'%atom0.get_potential_energy())
            for i in range(0, len(force0_qc)):
                for j in range(0, 3):
                    file.write('%24.18f'%delta[i][j])
                file.write('\n')
            file.write('\n')

        subprocess.run('cp CdSe_2.xyz CdSe.xyz', shell=True)

converged SCF energy = -114.918531116387
--------------- RKS gradients ---------------
         x                y                z
0 Se     0.0244807625    -0.0000000000    -0.0000000000
1 Se    -0.0244807624    -0.0000000000    -0.0000000000
2 Cd    -0.0000000000     0.0070607042     0.0000000000
3 Cd    -0.0000000000    -0.0070607042     0.0000000000
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 Se     0.0244807625    -0.0000000000    -0.0000000000
1 Se    -0.0244807624    -0.0000000000    -0.0000000000
2 Cd    -0.0000000000     0.0070607042     0.0000000000
3 Cd    -0.0000000000    -0.0070607042     0.0000000000
----------------------------------------------
converged SCF energy = -114.923530367731
--------------- RKS gradients ---------------
         x                y                z
0 Se     0.0028918839     0.0000000000     0.0000000000
1 Se    -0.0028918837     0.0000000000     0.00

2022-05-05 17:44:01.168563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-05 17:44:01.168604: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
/home/jingheng/venv/lib/python3.9/site-packages/deepmd/utils/data_system.py:158: UserWarning: system ./raw/ required batch size is larger than the size of the dataset raw/set.000 (8 > 3)
  warnings.warn("system %s required batch size is larger than the size of the dataset %s (%d > %d)" % \
2022-05-05 17:44:04.785771: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Ne

optimizer.py 77: [[-2.0118679  0.         0.       ]
 [ 2.0118679  0.         0.       ]
 [ 0.         1.6205108  0.       ]
 [ 0.        -1.6205108  0.       ]]
       Step     Time          Energy          fmax
LBFGS:    0 17:45:54    -3126.096629         0.874810
LBFGS:    1 17:45:54    -3126.134481         1.007578
LBFGS:    2 17:45:54    -3125.964238         0.151275
LBFGS:    3 17:45:54    -3125.958467         0.062675
LBFGS:    4 17:45:54    -3125.958181         0.037149
LBFGS:    5 17:45:54    -3125.958232         0.037243
LBFGS:    6 17:45:54    -3125.972002         0.439318
LBFGS:    7 17:45:54    -3125.959043         0.077608
LBFGS:    8 17:45:54    -3125.959443         0.040249
LBFGS:    9 17:45:54    -3125.959414         0.001089
LBFGS:   10 17:45:54    -3125.959480         0.181642
LBFGS:   11 17:45:54    -3125.957663         0.367483
LBFGS:   12 17:45:54    -3125.959229         0.154001
LBFGS:   13 17:45:54    -3125.959336         0.060363
LBFGS:   14 17:45:54    -3125.9

2022-05-05 17:46:27.499262: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-05 17:46:27.499302: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
/home/jingheng/venv/lib/python3.9/site-packages/deepmd/utils/data_system.py:158: UserWarning: system ./raw/ required batch size is larger than the size of the dataset raw/set.000 (8 > 4)
  warnings.warn("system %s required batch size is larger than the size of the dataset %s (%d > %d)" % \
2022-05-05 17:46:31.130929: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Ne

optimizer.py 77: [[-2.0118679  0.         0.       ]
 [ 2.0118679  0.         0.       ]
 [ 0.         1.6205108  0.       ]
 [ 0.        -1.6205108  0.       ]]
       Step     Time          Energy          fmax
LBFGS:    0 17:48:35    -3126.792206         0.735591
LBFGS:    1 17:48:35    -3126.813802         0.834282
LBFGS:    2 17:48:35    -3126.716474         0.143921
LBFGS:    3 17:48:35    -3126.715133         0.076105
LBFGS:    4 17:48:35    -3126.715380         0.078009
LBFGS:    5 17:48:35    -3126.684981         0.684014
LBFGS:    6 17:48:35    -3126.822173         1.453398
LBFGS:    7 17:48:35    -3126.667815         0.696828
LBFGS:    8 17:48:35    -3126.651737         0.678901
LBFGS:    9 17:48:35    -3126.606777         0.981211
LBFGS:   10 17:48:35    -3126.537945         0.277323
LBFGS:   11 17:48:35    -3126.523486         0.129080
LBFGS:   12 17:48:35    -3126.520179         0.024394
LBFGS:   13 17:48:35    -3126.520114         0.005048
LBFGS:   14 17:48:35    -3126.5

2022-05-05 17:49:07.749424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-05 17:49:07.749465: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
/home/jingheng/venv/lib/python3.9/site-packages/deepmd/utils/data_system.py:158: UserWarning: system ./raw/ required batch size is larger than the size of the dataset raw/set.000 (8 > 5)
  warnings.warn("system %s required batch size is larger than the size of the dataset %s (%d > %d)" % \
2022-05-05 17:49:11.306438: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Ne

optimizer.py 77: [[-2.0118679  0.         0.       ]
 [ 2.0118679  0.         0.       ]
 [ 0.         1.6205108  0.       ]
 [ 0.        -1.6205108  0.       ]]
       Step     Time          Energy          fmax
LBFGS:    0 17:51:30    -3126.678052         0.930857
LBFGS:    1 17:51:30    -3126.704595         1.049916
LBFGS:    2 17:51:30    -3126.588579         0.511339
LBFGS:    3 17:51:30    -3126.597538         0.524077
LBFGS:    4 17:51:30    -3126.421550         1.490706
LBFGS:    5 17:51:30    -3126.565155         2.664014
LBFGS:    6 17:51:30    -3126.264160         1.450597
LBFGS:    7 17:51:30    -3126.143818         1.097546
LBFGS:    8 17:51:30    -3125.973741         0.529000
LBFGS:    9 17:51:30    -3125.955699         0.068672
LBFGS:   10 17:51:30    -3125.954967         0.017075
LBFGS:   11 17:51:30    -3125.954927         0.001433
LBFGS:   12 17:51:30    -3125.954926         0.000155
converged SCF energy = -114.91715354082
--------------- RKS gradients ---------------

2022-05-05 17:52:04.060563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-05 17:52:04.060606: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
/home/jingheng/venv/lib/python3.9/site-packages/deepmd/utils/data_system.py:158: UserWarning: system ./raw/ required batch size is larger than the size of the dataset raw/set.000 (8 > 6)
  warnings.warn("system %s required batch size is larger than the size of the dataset %s (%d > %d)" % \
2022-05-05 17:52:07.649118: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Ne

optimizer.py 77: [[-2.0118679  0.         0.       ]
 [ 2.0118679  0.         0.       ]
 [ 0.         1.6205108  0.       ]
 [ 0.        -1.6205108  0.       ]]
       Step     Time          Energy          fmax
LBFGS:    0 17:54:41    -3126.825065         1.331807
LBFGS:    1 17:54:41    -3126.878501         1.474650
LBFGS:    2 17:54:41    -3126.600357         0.973665
LBFGS:    3 17:54:41    -3126.632252         0.912800
LBFGS:    4 17:54:41    -3126.773376         0.371599
LBFGS:    5 17:54:41    -3126.725634         1.039815
LBFGS:    6 17:54:41    -3126.768203         0.320994
LBFGS:    7 17:54:41    -3126.774995         0.048408
LBFGS:    8 17:54:41    -3126.774594         0.003615
LBFGS:    9 17:54:41    -3126.774592         0.000036
converged SCF energy = -114.906556876198
--------------- RKS gradients ---------------
         x                y                z
0 Se     0.0047311501    -0.0000000000    -0.0000000000
1 Se    -0.0047311691    -0.0000000000    -0.0000000000
2 C

ValueError: cannot reshape array of size 12 into shape (6,3)

In [ ]:
a,b,c = data.read_xyz('CdSe_2.xyz',index=1,output='regular')
print(b)

In [ ]:
mo = np.loadtxt('water.xyz',usecols=(1,2,3),skiprows=10)
# print(mo[1])
atoms =  ''
for i in range(0, len(b)):
    _mo = ''
    for j in range(0, 3):
        _mo += str(mo[i][j]) + '  '
    atoms += b[i] + '  ' + _mo + '\n'
print(atoms)

In [35]:
from pyscf import gto, scf, geomopt
from pyscf.data.nist import BOHR, HARTREE2EV
mol = gto.Mole(atom=atoms, basis='6-31g', spin=0).build()
mf = scf.RHF(mol)
mf.max_cycle = 500
mf.kernel()
print(mf.e_tot * HARTREE2EV)
mol_eq = mf.Gradients().optimizer(solver='berny').kernel()

converged SCF energy = -150.91286083677
-4106.548111611891


In [3]:
from ase.io import read
from ase.optimize import LBFGS
a = read('./CdSe_2.xyz', index=':')[0]
a.calc=DP(model='graph-compress.pb')
opt=LBFGS(a)
opt.run()

       Step     Time          Energy          fmax
LBFGS:    0 17:41:03    -3126.814622         1.367256
LBFGS:    1 17:41:03    -3126.870969         1.515481
LBFGS:    2 17:41:03    -3126.580287         0.990453
LBFGS:    3 17:41:03    -3126.613426         0.931764
LBFGS:    4 17:41:03    -3126.785507         0.677808
LBFGS:    5 17:41:03    -3126.500476         1.278227
LBFGS:    6 17:41:03    -3126.360157         1.931378
LBFGS:    7 17:41:03    -3126.475493         1.869027
LBFGS:    8 17:41:03    -3126.921155         1.130549
LBFGS:    9 17:41:03    -3126.855139         0.581809
LBFGS:   10 17:41:03    -3126.954638         1.112587
LBFGS:   11 17:41:03    -3127.076942         1.564375
LBFGS:   12 17:41:03    -3126.948635         1.115520
LBFGS:   13 17:41:03    -3126.835143         0.515549
LBFGS:   14 17:41:03    -3126.889379         1.151211
LBFGS:   15 17:41:03    -3126.783298         0.687865
LBFGS:   16 17:41:03    -3126.756730         0.161381
LBFGS:   17 17:41:03    -3126.7

True

In [3]:
pwd

'/home/jingheng/ML_QC_GEOM_OPT/test/CdSe/2'